In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import sys
import re
import sklearn.preprocessing
import warnings
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Suppress TensorFlow performance warnings and future warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
tf.get_logger().setLevel('ERROR')

In [5]:
# --- CONFIGURATION ---
DATASET_FILE = 'final_ml_features_normalized.csv' 
TARGET_COLUMN = 'Median_Housing_Value_2023' 
RANDOM_STATE = 1738
TRAIN_SIZE = 0.7  
VAL_SIZE = 0.15   
TEST_SIZE = 0.15  

# =========================================================================
# COMPLETE REPRODUCIBILITY SETUP
# =========================================================================

def set_random_seeds(seed=1738):
    """Set all random seeds for complete reproducibility"""
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    tf.config.experimental.enable_op_determinism()

set_random_seeds(RANDOM_STATE)

# 1. BASE FEATURE DEFINITION (Lagged and Pruned for Leakage) - 16 columns
BASE_NON_VIBE_COLS = [
    'Total_Population_2021', 'Median_Household_Income_2021', 
    'Owner_Occupied_Units_2021', 'Bachelors_Degree_Count_2021', 'Masters_Degree_Count_2021', 
    'Unemployed_Count_2021', 'Unemployment_Rate_2021', 'Bachelors_Or_Higher_Rate_2021', 
    'Total_Population_2022', 'Median_Household_Income_2022', 
    'Owner_Occupied_Units_2022', 'Bachelors_Degree_Count_2022', 'Masters_Degree_Count_2022', 
    'Unemployed_Count_2022', 'Unemployment_Rate_2022', 'Bachelors_Or_Higher_Rate_2022'
]

# 2. VIBE FEATURE DEFINITION (Full list)
VIBE_COLS_ALL = [
    'fortune', 'sbs', 'count', 
    'water_0', 'water_beach', 'water_coastline', 'water_harbor_harbour', 'water_lake', 
    'water_marina', 'water_river',
    'n_restaurants_PER_10K', 'n_cuisine_PER_10K', 'n_cuisine_mexican_PER_10K', 
    'n_cuisine_pizza_PER_10K', 'n_cuisine_american_PER_10K', 'n_cuisine_japanese_PER_10K', 
    'n_cuisine_thai_PER_10K', 'n_cuisine_sushi_PER_10K', 'n_cuisine_italian_PER_10K', 
    'n_cuisine_chinese_PER_10K', 'n_cuisine_korean_PER_10K', 'n_cuisine_burger_PER_10K', 
    'n_cuisine_mediterranean_PER_10K', 'n_cuisine_breakfast_PER_10K', 'n_cuisine_seafood_PER_10K', 
    'n_cuisine_asian_PER_10K', 'n_cuisine_indian_PER_10K', 'n_cuisine_barbecue_PER_10K', 
    'n_cuisine_sandwich_PER_10K', 'n_cuisine_steak_house_PER_10K', 'n_cuisine_vietnamese_PER_10K', 
    'n_cuisine_noodle_PER_10K', 'n_cuisine_chicken_PER_10K', 'n_cuisine_ramen_PER_10K', 
    'n_cuisine_french_PER_10K', 'n_cuisine_pasta_PER_10K', 'n_cuisine_filipino_PER_10K', 
    'n_cuisine_coffee_shop_PER_10K', 'n_cuisine_salad_PER_10K', 'n_cuisine_pancake_PER_10K', 
    'n_cuisine_peruvian_PER_10K', 'n_cuisine_hawaiian_PER_10K', 'n_cuisine_greek_PER_10K', 
    'n_cuisine_regional_PER_10K', 'n_cuisine_poke_PER_10K', 'n_cuisine_lebanese_PER_10K', 
    'n_cuisine_kebab_PER_10K', 'n_cuisine_fish_PER_10K', 'n_cuisine_spanish_PER_10K', 
    'n_cuisine_diner_PER_10K', 'n_cuisine_tacos_PER_10K', 'n_cuisine_dessert_PER_10K', 
    'n_cuisine_deli_PER_10K', 'n_cuisine_ice_cream_PER_10K', 'n_cuisine_brazilian_PER_10K', 
    'n_cuisine_middle_eastern_PER_10K', 'n_cuisine_tex-mex_PER_10K', 'n_cuisine_lunch_PER_10K', 
    'n_cuisine_grill_PER_10K', 'n_cuisine_brunch_PER_10K', 'n_cuisine_southern_PER_10K', 
    'n_cuisine_italian_pizza_PER_10K', 'n_cuisine_latin_american_PER_10K', 'n_cuisine_wings_PER_10K', 
    'n_cuisine_german_PER_10K', 'n_cuisine_steak_PER_10K', 'n_cuisine_bagel_PER_10K', 
    'n_cuisine_tapas_PER_10K', 'n_cuisine_fish_and_chips_PER_10K', 'n_cuisine_international_PER_10K', 
    'n_cuisine_jamaican_PER_10K', 'n_cuisine_turkish_PER_10K', 'n_cuisine_cajun_PER_10K', 
    'n_cuisine_soup_PER_10K', 'n_cuisine_buffet_PER_10K', 'n_cuisine_donut_PER_10K', 
    'n_cuisine_mongolian_grill_PER_10K', 'n_cuisine_caribbean_PER_10K', 'n_cuisine_hot_dog_PER_10K', 
    'n_cuisine_bar_and_grill_PER_10K', 'n_venues_PER_10K', 'n_venue_type_PER_10K', 
    'n_venue_bar_PER_10K', 'n_venue_fitness_centre_PER_10K', 'n_venue_theatre_PER_10K', 
    'n_venue_sports_centre_PER_10K', 'n_venue_cinema_PER_10K', 'n_venue_public_bookcase_PER_10K', 
    'n_venue_pub_PER_10K', 'n_venue_nightclub_PER_10K', 'n_venue_arts_centre_PER_10K', 
    'n_venue_stadium_PER_10K', 'n_venue_public_building_PER_10K', 'n_venue_casino_PER_10K', 
    'n_cafe_PER_10K', 'n_cafe_cuisine_PER_10K', 'n_cafe_brand_PER_10K', 
    'n_cafe_brand_Starbucks_PER_10K', 'n_cafe_brand_Dunkin\'_PER_10K', 'n_cafe_amenity_cafe_PER_10K', 
    'n_cafe_amenity_fast_food_PER_10K', 'n_cafe_cuisine_coffee_shop_PER_10K', 'n_cafe_cuisine_bubble_tea_PER_10K', 
    'n_cafe_cuisine_donut_PER_10K', 'n_cafe_cuisine_sandwich_PER_10K', 'n_cafe_cuisine_breakfast_PER_10K', 
    'n_cafe_cuisine_american_PER_10K', 'n_cafe_cuisine_coffee_PER_10K', 'n_cafe_cuisine_pastry_PER_10K', 
    'n_cafe_cuisine_tea_PER_10K', 'n_cafe_cuisine_dessert_PER_10K', 'n_cafe_cuisine_bagel_PER_10K', 
    'n_cafe_cuisine_ice_cream_PER_10K', 'n_cafe_brand_Tim_Hortons_PER_10K', 'n_cafe_brand_Dutch_Bros._Coffee_PER_10K', 
    'amenity_arts_centre_PER_10K', 'amenity_bar_PER_10K', 'amenity_community_centre_PER_10K', 
    'amenity_concert_hall_PER_10K', 'amenity_music_venue_PER_10K', 'amenity_nightclub_PER_10K', 
    'amenity_parking_PER_10K', 'amenity_pub_PER_10K', 'amenity_school_PER_10K', 
    'amenity_theatre_PER_10K', 'leisure_park_PER_10K', 'leisure_stadium_PER_10K', 
    'tourism_artwork_PER_10K', 'tourism_attraction_PER_10K', 'tourism_gallery_PER_10K', 
    'tourism_museum_PER_10K', 'tourism_picnic_site_PER_10K', 'tourism_viewpoint_PER_10K', 
    'tourism_yes_PER_10K', 'arts_theatre_events_PER_10K', 'music_events_PER_10K', 
    'other_events_PER_10K', 'sports_events_PER_10K', 'arts_theatre_venues_PER_10K', 
    'music_venues_PER_10K', 'other_venues_PER_10K', 'sports_venues_PER_10K'
]

# 3. COMBINED VIBE FEATURE SET
FULL_VIBE_COLS = BASE_NON_VIBE_COLS + VIBE_COLS_ALL

# Define Categorical Features
CATEGORICAL_COLS = [
    'water_0', 'water_beach', 'water_coastline', 'water_harbor_harbour', 
    'water_lake', 'water_marina', 'water_river'
]

In [6]:
# =========================================================================
# 1. DATA PREPARATION AND SPLITTING
# =========================================================================

def load_and_split_data(file_path, target_col, feature_cols_subset, global_cat_cols, train_size, val_size, test_size):
    """Loads data, filters duplicates, filters NaNs, and splits based on a specific feature subset."""
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: Data file not found at {file_path}. Please check path.")
        sys.exit(1)
        
    # --- 1a. Initial Filtering and Target/Population Check ---
    merge_key_col = [col for col in df.columns if 'Merge_Key' in col][0]
    pop_col = [col for col in df.columns if re.search(r'Total_Population', col, re.IGNORECASE)][0]
    df[pop_col] = pd.to_numeric(df[pop_col], errors='coerce')
    
    # --- 1b. Handle Duplicate Cities by Max Population ---
    initial_len = len(df)
    df = df.sort_values(by=pop_col, ascending=False).drop_duplicates(subset=[merge_key_col], keep='first').copy()
    print(f"Duplicate City Entries Removed: {initial_len - len(df)} rows dropped.")
    
    # --- 1c. Feature/Target Separation and NaN Target Removal ---
    required_cols = feature_cols_subset + [target_col]
    df_filtered = df[df.columns.intersection(required_cols)].copy()
    
    # Filter rows missing the target value
    df_filtered = df_filtered.dropna(subset=[target_col])
    
    X = df_filtered.drop(target_col, axis=1, errors='ignore')
    y = df_filtered[target_col]
    
    # --- 1d. DYNAMIC FEATURE IDENTIFICATION ---
    
    # 1. Identify which categorical columns are actually present in this model's X subset
    actual_cat_cols = [col for col in global_cat_cols if col in X.columns]
    
    # 2. Identify numerical columns present in this subset
    actual_num_cols = [col for col in X.columns if col not in actual_cat_cols and X[col].dtype != 'object']
    
    # 3. Final Imputation
    for col in actual_num_cols:
        X[col] = X[col].fillna(X[col].median())
    for col in actual_cat_cols:
        X[col] = X[col].fillna('missing') 
    
    # Ensure X only contains the columns used in the preprocessor
    X = X[actual_num_cols + actual_cat_cols]

    # --- 1e. Final Split ---
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, random_state=RANDOM_STATE
    )
    
    relative_val_size = val_size / (train_size + val_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=relative_val_size, random_state=RANDOM_STATE
    )
    
    print(f"Data Loaded and Split (Actual Features: {len(actual_num_cols) + len(actual_cat_cols)}):")
    print(f"  Training Set Size: {len(X_train)}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test, actual_num_cols, actual_cat_cols

In [7]:
# =========================================================================
# 2. PREPROCESSOR DEFINITION 
# =========================================================================

def define_preprocessor(numerical_features, categorical_features):
    """Defines the ColumnTransformer pipeline for scaling and encoding."""
    
    numerical_pipeline = StandardScaler()
    categorical_pipeline = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features),
            ('cat', categorical_pipeline, categorical_features)
        ],
        remainder='passthrough'
    )
    return preprocessor

In [8]:
# =========================================================================
# 3. NEURAL NETWORK MODELING (TensorFlow/Keras)
# =========================================================================

def build_mlp_model(input_shape, hidden_layers, activation='relu', dropout_rate=0.2, learning_rate=0.001):
    """Creates a sequential Multi-Layer Perceptron (MLP) model for regression."""
    # Reset seeds before building each model
    set_random_seeds(RANDOM_STATE)
    
    model = Sequential()
    
    # Input Layer and First Hidden Layer
    model.add(Dense(hidden_layers[0], activation=activation, input_shape=(input_shape,),
                    kernel_initializer=tf.keras.initializers.glorot_uniform(seed=RANDOM_STATE),
                    bias_initializer='zeros'))
    model.add(Dropout(dropout_rate, seed=RANDOM_STATE))
    
    # Additional Hidden Layers
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation,
                       kernel_initializer=tf.keras.initializers.glorot_uniform(seed=RANDOM_STATE),
                       bias_initializer='zeros'))
        model.add(Dropout(dropout_rate, seed=RANDOM_STATE))
        
    # Output Layer
    model.add(Dense(1,
                   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=RANDOM_STATE),
                   bias_initializer='zeros')) 
    
    # Compile the model with fixed optimizer
    optimizer = tf.keras.optimizers.legacy.Adam(
        learning_rate=learning_rate,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False
    )
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', tf.keras.metrics.R2Score()])
    
    return model

def run_hyperparameter_search(X_train, y_train, X_val, y_val, num_features, model_name):
    """Performs a custom grid search over predefined NN architecture and hyperparameters."""
    
    # Define the search space
    layer_configs = [(256, 128, 64), (128, 64), (64, 32), (256, 128)]
    learning_rates = [0.01, 0.001, 0.0001]
    batch_sizes = [32, 64, 128]
    dropout_rates = [0.1, 0.3]
    
    best_r2 = -float('inf')
    best_params = {}
    best_model = None
    
    print(f"\n--- Starting NN Grid Search for {model_name} ---")
    
    # Callbacks for better training
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    total_runs = len(layer_configs) * len(learning_rates) * len(batch_sizes) * len(dropout_rates)
    run_count = 0
    
    for layers in layer_configs:
        for lr in learning_rates:
            for bs in batch_sizes:
                for drop in dropout_rates:
                    run_count += 1
                    
                    # Reset everything for each model
                    tf.keras.backend.clear_session()
                    set_random_seeds(RANDOM_STATE)
                    
                    # 1. Build Model
                    model = build_mlp_model(input_shape=num_features, hidden_layers=layers, learning_rate=lr, dropout_rate=drop)
                    
                    # 2. Train Model
                    history = model.fit(
                        X_train, y_train, validation_data=(X_val, y_val), 
                        epochs=100, batch_size=bs, verbose=0, 
                        callbacks=[early_stopping, reduce_lr],
                        shuffle=False
                    )

                    # 3. Evaluate on Validation Set
                    val_metrics = model.evaluate(X_val, y_val, verbose=0)
                    val_r2 = val_metrics[-1]
                    
                    # 4. Track Best Model
                    if val_r2 > best_r2:
                        best_r2 = val_r2
                        best_params = {'hidden_layer_sizes': layers, 'learning_rate': lr, 'batch_size': bs, 'dropout_rate': drop}
                        best_model = model 

    print("\n--- Grid Search Complete ---")
    return best_model, best_params, best_r2

def evaluate_test_predictions(y_test_pred_scaled, y_test_true_scaled, y_scaler):
    """Calculates and returns standard regression metrics after inverse-transforming predictions."""
    
    # Inverse transform predictions and true values back to original dollar scale
    y_pred = y_scaler.inverse_transform(y_test_pred_scaled).flatten()
    y_true = y_scaler.inverse_transform(y_test_true_scaled).flatten() 

    if y_pred.shape != y_true.shape:
        print(f"ERROR: Prediction shape ({y_pred.shape}) does not match true target shape ({y_true.shape}). Cannot evaluate.")
        return {'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan}

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {'RMSE': np.sqrt(mse), 'MAE': mae, 'R2': r2}

def print_evaluation_summary(model_name, metrics, best_params):
    """Formats and prints the final model results."""
    print(f"\n--- {model_name} Test Set Evaluation ---")
    print("=" * 50)
    print(f"Optimal Architecture: {best_params['hidden_layer_sizes']}, LR={best_params['learning_rate']}, Drop={best_params['dropout_rate']}")
    print(f"Metric:              Value (in Original Scale)")
    print(f"--------------------------------------------------")
    print(f"Root Mean Squared Error (RMSE): {metrics['RMSE']:.2f}") 
    print(f"Mean Absolute Error (MAE):     {metrics['MAE']:.2f}")
    print(f"Coefficient of Determination (R²): {metrics['R2']:.4f}")
    print("=" * 50)

In [9]:
# =========================================================================
# 4. EXECUTION FLOW
# =========================================================================

def execute_model_run(feature_subset, model_name):
    """Executes the full preprocessing, training, and evaluation for a given feature subset."""
    
    print(f"\n\n=======================================================")
    print(f"MODEL RUN: {model_name}")
    print(f"=======================================================")
    
    # --- 1. Load and Split Data ---
    X_train, X_val, X_test, y_train, y_val, y_test, actual_num_cols, actual_cat_cols = load_and_split_data(
        DATASET_FILE, TARGET_COLUMN, feature_subset, CATEGORICAL_COLS, TRAIN_SIZE, VAL_SIZE, TEST_SIZE
    )
    
    # --- 2. Target Scaling ---
    y_scaler = sklearn.preprocessing.MinMaxScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
    y_val_scaled = y_scaler.transform(y_val.values.reshape(-1, 1))
    y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))
    
    # --- 3. Preprocessing Pipeline ---
    preprocessor = define_preprocessor(actual_num_cols, actual_cat_cols)
    
    print(f"Fitting and transforming features (Numeric: {len(actual_num_cols)}, Categorical: {len(actual_cat_cols)})...")
    X_train_proc = preprocessor.fit_transform(X_train)
    X_val_proc = preprocessor.transform(X_val)
    X_test_proc = preprocessor.transform(X_test)
    
    # --- 4. Type Conversion for TensorFlow ---
    X_train_proc = X_train_proc.astype(np.float32)
    X_val_proc = X_val_proc.astype(np.float32)
    X_test_proc = X_test_proc.astype(np.float32)
    
    y_train_proc = y_train_scaled.astype(np.float32)
    y_val_proc = y_val_scaled.astype(np.float32)
    y_test_proc = y_test_scaled.astype(np.float32) 
    
    num_features = X_train_proc.shape[1]
    print(f"Features (Numeric+Encoded): {num_features}")
    
    # --- 5. Hyperparameter Search and Training ---
    best_model, best_params, best_val_r2 = run_hyperparameter_search(
        X_train_proc, y_train_proc, X_val_proc, y_val_proc, num_features, model_name
    )

    # --- 6. Final Evaluation on Test Set ---
    if best_model:
        y_test_pred_scaled = best_model.predict(X_test_proc, verbose=0)
        
        test_metrics = evaluate_test_predictions(y_test_pred_scaled, y_test_proc, y_scaler)
        
        print_evaluation_summary(model_name, test_metrics, best_params)
        
        return best_model, best_params, test_metrics
    else:
        print(f"ERROR: {model_name} failed to find a best model.")
        return None, None, None


def main():
    """Main function to run both Non-VIBE and VIBE models and compare results."""
    
    set_random_seeds(RANDOM_STATE)
    
    # --- 1. RUN NON-VIBE BASE MODEL ---
    # Enforce lagging: only use BASE_NON_VIBE_COLS
    
    model_non_vibe, params_non_vibe, metrics_non_vibe = execute_model_run(
        BASE_NON_VIBE_COLS, "NON-VIBE BASE MODEL"
    )
    
    # --- 2. RUN FULL VIBE MODEL ---
    # Use all features: BASE_NON_VIBE_COLS + VIBE_COLS
    
    model_vibe, params_vibe, metrics_vibe = execute_model_run(
        FULL_VIBE_COLS, "FULL VIBE MODEL"
    )
    
    # --- 3. FINAL PROJECT COMPARISON ---
    
    if metrics_non_vibe and metrics_vibe:
        print("\n\n######################################################")
        print("####### FINAL VIBE FEATURE IMPACT COMPARISON #########")
        print("######################################################")
        
        r2_non_vibe = metrics_non_vibe['R2']
        r2_vibe = metrics_vibe['R2']
        
        if r2_vibe > r2_non_vibe:
            improvement = (r2_vibe - r2_non_vibe) / r2_non_vibe
            print(f"CONCLUSION: VIBE Features IMPROVED Predictive Power.")
            print(f"Baseline R² (Non-VIBE): {r2_non_vibe:.4f}")
            print(f"VIBE Model R²:          {r2_vibe:.4f} (+{improvement:.1%})")
        else:
            print(f"CONCLUSION: VIBE Features did NOT significantly improve R².")
            print(f"Baseline R² (Non-VIBE): {r2_non_vibe:.4f}")
            print(f"VIBE Model R²:          {r2_vibe:.4f}")
            
        print("\nRMSE Comparison:")
        print(f"Baseline RMSE: ${metrics_non_vibe['RMSE']:.2f}")
        print(f"VIBE Model RMSE: ${metrics_vibe['RMSE']:.2f}")
    

if __name__ == "__main__":
    tf.keras.backend.clear_session()
    set_random_seeds(RANDOM_STATE)
    main()



MODEL RUN: NON-VIBE BASE MODEL
Duplicate City Entries Removed: 46 rows dropped.
Data Loaded and Split (Actual Features: 16):
  Training Set Size: 605
Fitting and transforming features (Numeric: 16, Categorical: 0)...
Features (Numeric+Encoded): 16

--- Starting NN Grid Search for NON-VIBE BASE MODEL ---

--- Grid Search Complete ---

--- NON-VIBE BASE MODEL Test Set Evaluation ---
Optimal Architecture: (256, 128, 64), LR=0.01, Drop=0.1
Metric:              Value (in Original Scale)
--------------------------------------------------
Root Mean Squared Error (RMSE): 143621.49
Mean Absolute Error (MAE):     106303.38
Coefficient of Determination (R²): 0.7047


MODEL RUN: FULL VIBE MODEL
Duplicate City Entries Removed: 46 rows dropped.
Data Loaded and Split (Actual Features: 158):
  Training Set Size: 605
Fitting and transforming features (Numeric: 151, Categorical: 7)...
Features (Numeric+Encoded): 164

--- Starting NN Grid Search for FULL VIBE MODEL ---

--- Grid Search Complete ---

--